# I. Введение логарифмического барьера, выпуклость задачи.

## 1. Критерий выпуклости второго порядка

Optimal allocation with resource constraint:
$$f_{i}\left(x_{i}\right) = a_{i}e^{x_{i}}, a_{i} > 0, i \in \{ 1, ..., n\}$$

Также ограничение бюджета:
$$\sum^{n}_{i=1}x_{i} = b, b \in R$$

Дополнительное ограничение:
$$x_i \geq \frac{b}{2n} $$

Задача минимизации:
$$\sum^{n}_{i=1}a_{i}e^{x_{i}} \rightarrow min, a_{i} > 0, i \in \{ 1, ..., n\}$$

Задача с ограничениями:
$$\sum^{n}_{i=1}a_{i}e^{x_{i}} \rightarrow min, a_{i} > 0, i \in \{ 1, ..., n\},$$
$$
S.t. \sum^{n}_{i=1}x_{i} = b, \

x_i \geq \frac{b}{2n}, i \in \{ 1, ..., n\},  \in R$$

Обозначим ограничение-неравенство через $g(x_i) = \frac{b}{2n} - x_i \leq 0$.

Введем логарифмический барьер:
$$
\phi(x) = -\frac{1}{t} \sum_{i=1}^{n} \ln(-g(x_i)) = -\frac{1}{t} \sum_{i=1}^{n} \ln( x_i - \frac{b}{2n}), где   t > 0 - фикс.
$$ 

Итоговая задача с барьером:

$$\sum^{n}_{i=1} \left( a_{i} e^{x_{i}} -\frac{1}{t} \ln( x_i - \frac{b}{2n}) \right) \rightarrow min, a_{i} > 0, i \in \{ 1, ..., n\}, t > 0$$
$$
S.t. \sum^{n}_{i=1}x_{i} = b, b  \in R$$

Для изучения выпуклости рассмотрим гессиан оптимизируемой функции:
$$H(x)_{i,j} = 0, i \neq j$$
$$H(x)_{i,j} = a_{i}e^{x_{i}} + \frac{1}{t(x_i - \frac{b}{2n})^2}, i = j$$
$$\text{При } a_{i} > 0 \text{ и } t > 0 \text{ очевидно, что } H(x)_{i,i} > 0, \text{ так как } \forall x \in R: e^{x_{i}} > 0 \text{ и } \frac{1}{(x_i - \frac{b}{2n})^2} > 0$$
Матрица такого вида является положительно определенной.

Так же рассмотрим знакоопределенность гессиана ограничения:
$$H(x) = \textbf{0}$$ 

Таким образом, рассматриваемая барьерная задача является выпуклой.

# II. Решение тестовых примеров

In [2]:
import numpy as np
import cvxpy as cp
import time
import polars as pl
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [22]:
d=pl.read_csv("./0_01.csv")

In [23]:
columns_to_parse = ["a", "cvxpy_x", "x_init", "newton_x", "newton_hist_y", "newton_iters_hist"]

d=d.with_columns([
    pl.col(col)
    .str.strip_chars("[]")
    .str.replace_all("\n", " ")
    .str.replace_all(r"\s+", " ")                      
    .str.strip_chars(" ")                             
    .str.split(" ")                                     
    .list.eval(
        pl.when(pl.element().str.strip_chars(" ") != "") 
        .then(pl.element().cast(pl.Float64))
        .otherwise(None),
        parallel=True
    )
    .alias(col)
   
    for col in columns_to_parse
])
d

,dim,a,b,cvxpy_x,cvxpy_y,cvxpy_t,x_init,mu,newton_x,newton_y,newton_t,newton_hist_y,newton_iters_hist
i64,i64,list[f64],f64,list[f64],f64,f64,list[f64],i64,list[f64],f64,f64,list[f64],list[f64]
0,10,"[5.728031, 4.123869, … 3.202003]",5.50868,"[0.275434, 0.275434, … 0.378602]",69.537908,0.016473,"[0.570959, 0.750641, … 0.537103]",2,"[0.27611, 0.277929, … 0.379767]",69.549568,0.061801,"[73.527029, 71.754103, … 69.549568]","[11.0, 7.0, … 5.0]"
1,10,"[5.728031, 4.123869, … 3.202003]",5.50868,"[0.275434, 0.275434, … 0.378602]",69.537908,0.016473,"[0.570959, 0.750641, … 0.537103]",10,"[0.275473, 0.275676, … 0.387128]",69.539472,0.01768,"[73.527029, 70.066583, … 69.539472]","[11.0, 8.0, … 2.0]"
2,10,"[5.728031, 4.123869, … 3.202003]",5.50868,"[0.275434, 0.275434, … 0.378602]",69.537908,0.016473,"[0.570959, 0.750641, … 0.537103]",50,"[0.282004, 0.295588, … 0.389209]",69.653196,0.011347,"[73.527029, 69.653196]","[11.0, 9.0]"
3,10,"[5.728031, 4.123869, … 3.202003]",5.50868,"[0.275434, 0.275434, … 0.378602]",69.537908,0.016473,"[0.570959, 0.750641, … 0.537103]",100,"[0.278812, 0.286754, … 0.3841995]",69.596571,0.012344,"[73.527029, 69.596571]","[11.0, 12.0]"
4,10,"[5.728031, 4.123869, … 3.202003]",5.50868,"[0.275434, 0.275434, … 0.378602]",69.537908,0.016473,"[0.367604, 0.435501, … 0.374071]",2,"[0.27611, 0.277929, … 0.379767]",69.549568,0.046368,"[73.527029, 71.754103, … 69.549568]","[9.0, 7.0, … 5.0]"
…,…,…,…,…,…,…,…,…,…,…,…,…,…
399995,100,"[8.868352, 4.717864, … 9.771052]",4.881492,"[0.024407, 0.024407, … 0.024407]",493.341466,0.00992,"[0.064983, 0.065731, … 0.056654]",100,"[0.024419, 0.02443, … 0.024418]",493.350956,0.034183,"[504.247845, 494.268595, 493.350956]","[11.0, 16.0, 11.0]"
399996,100,"[8.868352, 4.717864, … 9.771052]",4.881492,"[0.024407, 0.024407, … 0.024407]",493.341466,0.00992,"[0.051174, 0.072109, … 0.047502]",2,"[0.0244216, 0.024435, … 0.02442]",493.353059,0.074914,"[504.247845, 503.761495, … 493.353059]","[13.0, 4.0, … 4.0]"
399997,100,"[8.868352, 4.717864, … 9.771052]",4.881492,"[0.024407, 0.024407, … 0.024407]",493.341466,0.00992,"[0.051174, 0.072109, … 0.047502]",10,"[0.024414, 0.024421, … 0.024414]",493.347046,0.03748,"[504.247845, 500.258673, … 493.347046]","[13.0, 7.0, … 3.0]"


## 1. Генерация примеров

In [2]:
n_dims = range(10, 110, 10)
n_examples = 100
n_seeds = 100

In [3]:
raw_experiment_sheet = pl.DataFrame({"dim": n_dims}).select(
    pl.all().repeat_by(n_examples).flatten()
)

In [4]:
raw_experiment_sheet = raw_experiment_sheet.with_columns(
    pl.col("dim")
    .map_elements(
        lambda dim_: np.random.uniform(low=0.1, high=10, size=dim_),
        return_dtype=pl.List(float),
        returns_scalar=False,
    )
    .alias("a"),
    pl.col("dim")
    .map_elements(
        lambda dim_: np.random.uniform(low=0, high=10, size=1),
        return_dtype=float,
        returns_scalar=True,
    )
    .alias("b"),
)

In [5]:
raw_experiment_sheet

dim,a,b
i64,list[f64],f64
10,"[3.648221, 9.557395, … 1.27787]",2.193924
10,"[7.866477, 7.236956, … 4.962097]",1.746916
10,"[0.953317, 0.257531, … 5.030669]",6.675724
10,"[8.824198, 8.862705, … 8.242145]",1.89879
10,"[9.168859, 9.514291, … 5.679912]",8.708293
…,…,…
100,"[5.73979, 1.078335, … 8.80275]",0.043402
100,"[8.770381, 0.727317, … 0.45495]",5.468644
100,"[5.61154, 3.872917, … 1.015694]",6.661169


## 2. Решение солвером CVXPY

In [6]:
def solve_oawrc(a, b):

    a = np.asarray(a)
    n = a.shape[0]
    
    x = cp.Variable(n)

    objective = cp.Minimize(cp.sum(cp.multiply(a, cp.exp(x))))

    constraints = [cp.sum(x) - b == 0, x >= b / (2 * n)]

    prob = cp.Problem(objective, constraints)

    try:
        start_t = time.time()
        result = prob.solve()
        sovle_time = time.time() - start_t
    except:
        res = {"cvxpy_x": None, "cvxpy_y": None, "cvxpy_t": None}
    else:
        res = {"cvxpy_x": list(x.value), "cvxpy_y": result, "cvxpy_t": sovle_time}

    return res

In [7]:
raw_experiment_sheet = raw_experiment_sheet.with_columns(
    (
        pl.struct("a", "b")
        .map_elements(
            lambda x: solve_oawrc(**x),
            return_dtype=pl.Struct(
                {"cvxpy_x": pl.List(float), "cvxpy_y": float, "cvxpy_t": float}
            ),
        )
        .alias("cvxpy_out")
    )
).unnest("cvxpy_out")

In [8]:
raw_experiment_sheet

dim,a,b,cvxpy_x,cvxpy_y,cvxpy_t
i64,list[f64],f64,list[f64],f64,f64
10,"[3.648221, 9.557395, … 1.27787]",2.193924,"[0.109696, 0.109696, … 0.468692]",55.997499,0.023679
10,"[7.866477, 7.236956, … 4.962097]",1.746916,"[0.087346, 0.087346, … 0.087346]",64.042871,0.016259
10,"[0.953317, 0.257531, … 5.030669]",6.675724,"[0.533067, 1.841796, … 0.333786]",45.302588,0.013741
10,"[8.824198, 8.862705, … 8.242145]",1.89879,"[0.094939, 0.094939, … 0.094939]",74.346259,0.011155
10,"[9.168859, 9.514291, … 5.679912]",8.708293,"[0.435415, 0.435415, … 0.835365]",133.820821,0.009419
…,…,…,…,…,…
100,"[5.73979, 1.078335, … 8.80275]",0.043402,"[0.000217, 0.000217, … 0.000217]",484.770212,0.009479
100,"[8.770381, 0.727317, … 0.45495]",5.468644,"[0.027343, 0.027345, … 0.466399]",516.227522,0.009313
100,"[5.61154, 3.872917, … 1.015694]",6.661169,"[0.033306, 0.033306, … 0.033306]",518.12682,0.009668


## 3. Решение прямой задачи методом Ньютона 

In [9]:
def random_sum_bound(dim, b):
    rnd = np.random.uniform(size=dim)
    y = rnd / rnd.sum() * (b / 2)
    return list(y + b / (2 * dim))

In [10]:
raw_experiment_sheet = raw_experiment_sheet.select(
    pl.all().repeat_by(n_seeds).flatten()
).with_columns(
    pl.struct("dim", "b")
    .map_elements(lambda x: random_sum_bound(**x), return_dtype=pl.List(float))
    .alias("x_init")
)

In [11]:
raw_experiment_sheet = raw_experiment_sheet.with_columns(
        pl.lit([2, 10, 50, 100]).alias("mu")
    ).explode("mu")                      

raw_experiment_sheet = raw_experiment_sheet.drop_nans()
raw_experiment_sheet

dim,a,b,cvxpy_x,cvxpy_y,cvxpy_t,x_init,mu
i64,list[f64],f64,list[f64],f64,f64,list[f64],i64
10,"[3.648221, 9.557395, … 1.27787]",2.193924,"[0.109696, 0.109696, … 0.468692]",55.997499,0.023679,"[0.327512, 0.182852, … 0.275643]",2
10,"[3.648221, 9.557395, … 1.27787]",2.193924,"[0.109696, 0.109696, … 0.468692]",55.997499,0.023679,"[0.327512, 0.182852, … 0.275643]",10
10,"[3.648221, 9.557395, … 1.27787]",2.193924,"[0.109696, 0.109696, … 0.468692]",55.997499,0.023679,"[0.327512, 0.182852, … 0.275643]",50
10,"[3.648221, 9.557395, … 1.27787]",2.193924,"[0.109696, 0.109696, … 0.468692]",55.997499,0.023679,"[0.327512, 0.182852, … 0.275643]",100
10,"[3.648221, 9.557395, … 1.27787]",2.193924,"[0.109696, 0.109696, … 0.468692]",55.997499,0.023679,"[0.269399, 0.142283, … 0.112069]",2
…,…,…,…,…,…,…,…
100,"[3.379433, 1.499171, … 4.013805]",2.731074,"[0.013655, 0.013655, … 0.013655]",497.586446,0.009398,"[0.027842, 0.037638, … 0.017058]",100
100,"[3.379433, 1.499171, … 4.013805]",2.731074,"[0.013655, 0.013655, … 0.013655]",497.586446,0.009398,"[0.017348, 0.027547, … 0.039449]",2
100,"[3.379433, 1.499171, … 4.013805]",2.731074,"[0.013655, 0.013655, … 0.013655]",497.586446,0.009398,"[0.017348, 0.027547, … 0.039449]",10


In [12]:
def f(x, a):
    return np.dot(a, np.exp(x)).reshape(1,1)

def grad_f(x, a):
    return (a * np.exp(x)).reshape(1,-1)

def hessian_f(x, a):
    return np.diag(a * np.exp(x))

def barrier_f(x, a, b, t):
    return (t * np.dot(a, np.exp(x)) - np.sum(np.log(x - b / (2*len(a))))).reshape(1,1)

def barrier_grad_f(x, a, b, t):
    return (t * a * np.exp(x) - 1. / (x - b/(2*len(a)))).reshape(1,-1)

def barrier_hessian_f(x, a, b, t):
    return np.diag(t * a * np.exp(x) + 1. / (x - b / (2*len(a)))**2 )

def g(x, b):
    return np.array([np.sum(np.asarray(x)) - b]).reshape(1,1)

def grad_g(x):
    return np.ones(len(x)).reshape(1,-1)

def hessian_g(x):
    return np.zeros([len(x)]*2)

In [15]:
def armijo_line_search(x, a, b, t, dx, alpha0=1.0, beta=0.5, c=1e-4, max_iters=20):
    alpha = alpha0

    func_value = barrier_f(x, a, b, t)[0][0]
    gTd  = np.dot(barrier_grad_f(x, a, b, t), dx)
    
    for _ in range(max_iters):
        x_new = x + alpha * dx
        func_value_new = barrier_f(x_new, a, b, t)[0][0]

        if np.all(x_new >= b / (2*len(a))) and np.allclose(np.sum(x_new), b) and func_value_new < func_value + c * alpha * gTd:
            return alpha
        alpha *= beta
    
    return None

def solve_oawrc_newton(a, b, x_init, cvxpy_y, t, eps=0.01, max_iters=20, max_linesearch_iters=30, c=0.2, beta=0.5):

    x = np.asarray(x_init)
    a = np.asarray(a)
    
    for i in range(max_iters):
        grad_L = barrier_grad_f(x, a, b, t)
        hessian_L = barrier_hessian_f(x, a, b, t)

        hessian_L_full = np.block([
            [hessian_L, grad_g(x).T],
            [grad_g(x), np.zeros([1,1])]
        ])
        
        grad_L_full = np.concatenate([grad_L, g(x, b).reshape(1,-1)], axis = -1)
        d = np.linalg.solve(hessian_L_full, -grad_L_full.T)
        dx = d[:len(x),0]

        alpha = armijo_line_search(x, a, b, t, dx, alpha0=1.0, beta=beta, c=c, max_iters=max_linesearch_iters)
        if alpha is None:
            break

        x = x + alpha * dx

        curr_f = f(x, a)[0][0]
        if np.abs(curr_f - cvxpy_y) < eps:
            break

    return x.tolist(), i+1, curr_f

def barrier_method(a, b, x_init, cvxpy_y, mu=2, t_init=1, eps=0.001, max_centering_iters=20):
    print(len(a), mu)
    a = np.asarray(a)
    n = a.shape[0]
    x = x_init.copy()
    t = t_init

    sovle_time = time.time()
    newton_iters_hist, newton_hist_y = [], []

    while n/t >= eps:
        x, centering_iters, curr_y = solve_oawrc_newton(a, b, x, cvxpy_y, t, eps=eps, max_iters=max_centering_iters)

        newton_iters_hist.append(centering_iters)
        newton_hist_y.append(curr_y)

        t *= mu
    print("curr", curr_y, "cvx", cvxpy_y, curr_y-cvxpy_y, np.all(np.asarray(x) >= b / (2*len(a))), np.allclose(np.sum(x), b), newton_iters_hist)
    return {"newton_x": x, "newton_y": curr_y, "newton_t": time.time() - sovle_time, "newton_hist_y": newton_hist_y, "newton_iters_hist": newton_iters_hist}

In [ ]:
raw_experiment_sheet = raw_experiment_sheet.with_columns(
    (
        pl.struct("a", "b", "x_init", "cvxpy_y", "mu")
        .map_elements(
            lambda x: barrier_method(**x),
            return_dtype=pl.Struct(
                {"newton_x": pl.List(float), "newton_y": float, "newton_t": float, "newton_hist_y": pl.List(float), "newton_iters_hist": pl.List(int)}
            ),
        )
        .alias("newton_out")
    )
).unnest("newton_out")

In [42]:
raw_experiment_sheet

dim,a,b,cvxpy_x,cvxpy_y,cvxpy_t,x_init,mu,newton_x,newton_y,newton_t,newton_hist_y,newton_iters_hist
i64,list[f64],f64,list[f64],f64,f64,list[f64],i64,list[f64],f64,f64,list[f64],list[i64]
10,"[2.266699, 6.338006, … 9.679609]",0.536932,"[0.026847, 0.026847, … 0.026847]",48.285159,0.031559,"[0.062119, 0.075646, … 0.058166]",2,"[0.026935, 0.026869, … 0.02686]",48.286135,0.123808,"[49.286848, 49.221879, … 48.286135]","[9, 4, … 1]"
10,"[2.266699, 6.338006, … 9.679609]",0.536932,"[0.026847, 0.026847, … 0.026847]",48.285159,0.031559,"[0.062119, 0.075646, … 0.058166]",10,"[0.026889, 0.026859, … 0.026854]",48.285678,0.027595,"[49.286848, 48.845302, … 48.285678]","[9, 7, … 2]"
10,"[2.266699, 6.338006, … 9.679609]",0.536932,"[0.026847, 0.026847, … 0.026847]",48.285159,0.031559,"[0.062119, 0.075646, … 0.058166]",50,"[0.027135, 0.026918, … 0.026891]",48.288356,0.022317,"[49.286848, 48.436636, 48.288356]","[9, 9, 12]"
10,"[2.266699, 6.338006, … 9.679609]",0.536932,"[0.026847, 0.026847, … 0.026847]",48.285159,0.031559,"[0.062119, 0.075646, … 0.058166]",100,"[0.027022, 0.026854, … 0.026852]",48.285772,0.01519,"[49.286848, 48.363123, 48.285772]","[9, 9, 4]"
10,"[2.266699, 6.338006, … 9.679609]",0.536932,"[0.026847, 0.026847, … 0.026847]",48.285159,0.031559,"[0.067978, 0.03281, … 0.069219]",2,"[0.026935, 0.026869, … 0.02686]",48.286135,0.068592,"[49.286848, 49.221879, … 48.286135]","[9, 4, … 1]"
…,…,…,…,…,…,…,…,…,…,…,…,…
100,"[6.185764, 5.141971, … 7.544346]",7.533661,"[0.037668, 0.037668, … 0.037668]",547.174836,0.00946,"[0.106166, 0.053893, … 0.106945]",100,"[0.037687, 0.037691, … 0.037683]",547.184134,0.027852,"[564.188621, 548.079427, 547.184134]","[11, 17, 17]"
100,"[6.185764, 5.141971, … 7.544346]",7.533661,"[0.037668, 0.037668, … 0.037668]",547.174836,0.00946,"[0.083529, 0.084167, … 0.091702]",2,"[0.037671, 0.037672, … 0.037671]",547.176256,0.091363,"[564.188621, 562.7755, … 547.176256]","[12, 5, … 4]"
100,"[6.185764, 5.141971, … 7.544346]",7.533661,"[0.037668, 0.037668, … 0.037668]",547.174836,0.00946,"[0.083529, 0.084167, … 0.091702]",10,"[0.03767, 0.03767, … 0.037669]",547.175453,0.035969,"[564.188621, 554.939015, … 547.175453]","[12, 8, … 2]"


# III. Анализ зависимостей

In [ ]:
res_data = raw_experiment_sheet.clone()
res_data = (res_data
    .with_columns(pl.col("newton_hist_y").list.len().alias("iter_num"))
    .with_columns((pl.col("cvxpy_y") - pl.col("newton_y")).abs().alias("y_diff"))
    .select("dim", "mu", "newton_t", "iter_num", "y_diff")
    .group_by("dim", "mu")
    .agg(pl.all().mean().name.suffix("_mean"))
    .sort("dim", "mu")
)

# IV. Выводы